### Import basic libraries

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

### Load in the dataframe

In [2]:
df = pd.read_pickle('./main_df.pkl')
print(df.shape)
df.head()

(2213180, 14)


,virus,host,blastn,crisprdetect-2mismatch,gc_content,k25,k6-chebyshev,k6-kendalltau,k6-manhattan,piler-2mismatch,wish,group,group_code,y
0,NC_010363,NC_008527,60.8,0.0,0.021973,0.0,0.002122,0.398421,0.382144,0.0,-1.33553,Streptococcaceae,41,1
1,NC_010363,NC_002662,59.0,0.0,0.016709,0.0,0.001929,0.397773,0.377498,0.0,-1.33035,Streptococcaceae,41,1
2,NC_010363,NC_017949,59.0,0.0,0.020818,0.0,0.002088,0.396969,0.379686,0.0,-1.33341,Streptococcaceae,41,1
3,NC_010363,NC_017492,59.0,0.0,0.022209,0.0,0.002131,0.396148,0.380930,0.0,-1.33767,Streptococcaceae,41,1
4,NC_010363,NC_009004,59.0,0.0,0.020871,0.0,0.002088,0.397095,0.379834,0.0,-1.33341,Streptococcaceae,41,1


## Data preprocessing

### Get all of positive cases + the same number of negative cases randomly

In [3]:
# get all the positive cases
learning_df = df[df['y'] == 1]
# get the same amount of negative cases RANDOMLY
negative_learning_df = df[df['y'] == 0].sample(n=len(df[df['y'] == 1].index))

learning_df = pd.concat([learning_df, negative_learning_df], ignore_index=True)
print(learning_df.shape)
learning_df.head()

(33514, 14)


,virus,host,blastn,crisprdetect-2mismatch,gc_content,k25,k6-chebyshev,k6-kendalltau,k6-manhattan,piler-2mismatch,wish,group,group_code,y
0,NC_010363,NC_008527,60.8,0.0,0.021973,0.0,0.002122,0.398421,0.382144,0.0,-1.33553,Streptococcaceae,41,1
1,NC_010363,NC_002662,59.0,0.0,0.016709,0.0,0.001929,0.397773,0.377498,0.0,-1.33035,Streptococcaceae,41,1
2,NC_010363,NC_017949,59.0,0.0,0.020818,0.0,0.002088,0.396969,0.379686,0.0,-1.33341,Streptococcaceae,41,1
3,NC_010363,NC_017492,59.0,0.0,0.022209,0.0,0.002131,0.396148,0.380930,0.0,-1.33767,Streptococcaceae,41,1
4,NC_010363,NC_009004,59.0,0.0,0.020871,0.0,0.002088,0.397095,0.379834,0.0,-1.33341,Streptococcaceae,41,1


### Groups dristribution

NOTE: evening out the differences in group compositions will be added later

In [23]:
learning_df[['group','group_code']].value_counts()

group                                              group_code
Enterobacteriaceae                                 16            12734
Staphylococcaceae                                  40             4872
Pseudomonadaceae                                   34             2914
Streptococcaceae                                   41             2235
Bacillaceae                                        5              1534
Burkholderiaceae                                   9              1202
Vibrionaceae                                       46             1144
Listeriaceae                                       22              966
Propionibacteriaceae                               33              654
Mycobacteriaceae                                   25              641
Xanthomonadaceae                                   47              418
Lactobacillaceae                                   20              417
Clostridiaceae                                     13              308
Enterococcaceae

In [28]:
learning_df[learning_df['y'] == 1][['group', 'group_code']].value_counts()

group                                              group_code
Enterobacteriaceae                                 16            8419
Staphylococcaceae                                  40            3342
Pseudomonadaceae                                   34            1012
Streptococcaceae                                   41             782
Listeriaceae                                       22             624
Bacillaceae                                        5              454
Vibrionaceae                                       46             436
Burkholderiaceae                                   9              342
Helicobacteraceae                                  19             212
Propionibacteriaceae                               33             211
Mycobacteriaceae                                   25             156
Moraxellaceae                                      24             113
Lactobacillaceae                                   20              74
Xanthomonadaceae            

### Encode categorical values

*Note: encoding not needed because training is not based on virus and host names*

In [6]:
'''
transformed_data = pd.get_dummies(filled_df, columns=['virus', 'host'])
transformed_data
'''

"\ntransformed_data = pd.get_dummies(filled_df, columns=['virus', 'host'])\ntransformed_data\n"

### Extract X and y arrays

In [27]:
X = learning_df.drop(['virus', 'host', 'group', 'group_code', 'y'], axis=1).values
y = learning_df['y'].values

print(f'shape of X: {X.shape}')
print(f'len(y): {len(y)}')

shape of X: (33514, 9)
len(y): 33514


Get groups for LeaveOneGroupOut

In [30]:
groups = learning_df['group_code'].values

### test_train_split

In [ ]:
# SIMPLE SPLIT
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25)

### Feature scaling (if necessary) 

### Train the Random Forst Classifier

In [ ]:
# ONLY FOR SIMPLE SPLIT USE
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state = 0)
classifier.fit(X_train, y_train)

### Predict Test set results

In [ ]:
y_pred = classifier.predict(X_test)
print(np.concatenate((y_pred.reshape(len(y_pred),1), y_test.reshape(len(y_test),1)),1))

## Scoring the model

### Confusion Matrix

In [ ]:
from sklearn.metrics import confusion_matrix, accuracy_score
cm = confusion_matrix(y_test, y_pred)
print(cm)
accuracy_score(y_test, y_pred)

### ROC curve

In [ ]:
import sklearn.metrics as metrics
# calculate the fpr and tpr for all thresholds of the classification
probs = classifier.predict_proba(X_test)
preds = probs[:,1]
fpr, tpr, threshold = metrics.roc_curve(y_test, preds)
roc_auc = metrics.auc(fpr, tpr)

# method I: plt
import matplotlib.pyplot as plt
plt.title('Receiver Operating Characteristic')
plt.plot(fpr, tpr, 'b', label = 'AUC = %0.2f' % roc_auc)
plt.legend(loc = 'lower right')
plt.plot([0, 1], [0, 1],'r--')
plt.xlim([0, 1])
plt.ylim([0, 1])
plt.ylabel('True Positive Rate')
plt.xlabel('False Positive Rate')
plt.show()

*Note: conclusion: ROC curve is a bad method to evaluate this kind of data*

### Precision-recall curve

In [ ]:
#TODO implement this
from sklearn.metrics import PrecisionRecallDisplay
display = PrecisionRecallDisplay.from_predictions(y_test, y_pred, name="r_forest")
_ = display.ax_.set_title("r_forest (10 estimators) precision-recall curve")

In [ ]:
display = PrecisionRecallDisplay.from_estimator(
    classifier, X_test, y_test, name="r_forest"
)
_ = display.ax_.set_title("r_forest (10 estimators) Precision-Recall curve")

## F1 score

In [ ]:
print(cm)
accuracy_score(y_test, y_pred)

### Calculate precision and recall

In [ ]:
from sklearn.metrics import precision_score, recall_score
print('Precision is: ', precision_score(y_test, y_pred))
print('Recall is: ', recall_score(y_test, y_pred))

### F1 score for this model 

In [ ]:
from sklearn.metrics import f1_score
print(f'F1 is: {f1_score(y_test, y_pred)}')

*Note: the model still has a very good score*

*Ideas as to why:* 
1. *Now there is enough data to train the model - ca. 16,5k positive cases + 16,5k randomly chosen negative ones*
    * _to get learning_df closer to natural, a different method (learning for unbalanced set - ___stratified sampling___) will probably be needed_ 
2. *Still huge host and phage in-group similarity (hosts often with only a few nucleotide difference in genome) - not adressed!* 
3. *Randomly chosen 'good' negative cases?*

# Training using cross validation

In [6]:
# Leave one group out split
from sklearn.model_selection import LeaveOneGroupOut
logo = LeaveOneGroupOut()

### Using cross_val_score function

In [ ]:
# Leave one group out split
from sklearn.model_selection import LeaveOneGroupOut
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import RandomForestClassifier

# create logo cv procedure
logo = LeaveOneGroupOut()
# create model
model = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state=1)
# evaluate model
scores = cross_val_score(model, X, y, scoring='f1', cv=logo, groups=groups, n_jobs=-1)
# report performance
print(f'Mean F1 score: {np.mean(scores):.3f}, stdev: ({np.std(scores):.3f})')

Mean F1 score: 0.523, stdev: (0.338)


### Is the prediction replicable + does it work the way it's supposed to?

In [62]:
idx = learning_df[learning_df['group_code'] == 0].index
X_ref_train = learning_df.drop(index=idx).drop(['virus', 'host', 'group', 'group_code', 'y'], axis=1).values
X_ref_test = learning_df.iloc[idx,:].drop(['virus', 'host', 'group', 'group_code', 'y'], axis=1).values
y_ref_train = learning_df.drop(index=idx).iloc[:, -1]
y_ref_test = learning_df.iloc[idx,-1]

In [64]:
model = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state=1)
model.fit(X_ref_train, y_ref_train)
y_pred = model.predict(X_ref_test)

In [66]:
from sklearn.metrics import f1_score
print(f'F1 is: {f1_score(y_ref_test, y_pred)}')

F1 is: 0.6666666666666666


NOTE: the F1 score is the same in the results of the LeaveOneGroupOut cross validation for this group

# TODO: implement F1 + precision/recall curves for all estimators in cross validation

e.g. take estimator from cross_validate function + use from_estimator 

# Additional/old code

Explicit tqdm handling (better visualisaition)

*Watch out -* ***long!***

*This code is for LeaveOneOut* ***only!***

In [ ]:
'''
# FOR LEAVE ONE OUT ONLY!
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from tqdm import tqdm
# enumerate splits
y_true, y_pred = list(), list()
pbar = tqdm(total=len(X))

for train_ix, test_ix in cv.split(X):
    # split data
    X_train, X_test = X[train_ix, :], X[test_ix, :]
    y_train, y_test = y[train_ix], y[test_ix]
    # fit model
    model = RandomForestClassifier(n_estimators = 10, criterion = 'entropy', random_state=1)
    model.fit(X_train, y_train)
    # evaluate model
    yhat = model.predict(X_test)
    # store
    y_true.append(y_test[0])
    y_pred.append(yhat[0])
    pbar.update(1)
pbar.close()
# calculate accuracy
acc = accuracy_score(y_true, y_pred)
print(f'Accuracy: {acc:.3f}')
'''

  0%|          | 48/33514 [00:25<4:59:19,  1.86it/s]

Accuracy: 0.583
